In [72]:
### INITIALIZATION #############################

import pandas as pd
import os
import csv

############ Libraries ##############

from matplotlib import pyplot as plt
import numpy as np

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

epislon = 1e-20  # Define a small epsilon value for division by zero cases

def rmse(y_true, y_pred):
  return np.sqrt(mse(y_true, y_pred))

def mase(y_true, y_pred, y_baseline):
    # Calcula o MAE do modelo
    mae_pred = np.mean(np.abs(y_true - y_pred))
    # Calcula o MAE do modelo baseline Persistent Window (i.e., últimas h observações antes do teste)
    mae_naive = np.mean(np.abs(y_true - y_baseline))
    result = mae_pred/mae_naive
    return result

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_true - y_pred)/np.sum(y_true))
  else:
    return 100*(np.sum(y_true - y_pred)/(np.sum(y_true) + epislon))

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

def mcpm(rmse_result, mape_result, pocid_result):
  er_result = 100 - pocid_result

  A1 = (rmse_result * mape_result * np.sin((2*np.pi)/3))/2
  A2 = (mape_result * er_result * np.sin((2*np.pi)/3))/2
  A3 = (er_result * rmse_result * np.sin((2*np.pi)/3))/2
  total = A1 + A2 + A3
  return total

def znorm(x):
  if np.std(x) != 0: 
      x_znorm = (x - np.mean(x)) / np.std(x)
  else:
      x_znorm = (x - np.mean(x)) / (np.std(x) + epislon)
  return x_znorm

def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds

def baseline_mean(series, horizon):
  # como as séries são normalizadas, esse baseline irá retornar uma reta próxima de zero
  pred = np.repeat(np.mean(znorm(series[:-horizon])), horizon)
  return pred

def baseline_persistent(series, horizon):
  return np.repeat(znorm(series[-2*horizon:-horizon]).values[-1], horizon)

def baseline_persistent_window(series, horizon):
  subsequence = znorm(series[-horizon*2:-horizon]).values
  return subsequence

def baseline_persistent_windowR(series, horizon):
  subsequence2 = series[-horizon*2:-horizon].values
  return subsequence2

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df


def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados


In [4]:
######### ZERO SHOT

import pandas as pd  # requires: pip install pandas
import torch
from chronos import BaseChronosPipeline

# Model
# chronos-t5-tiny
# chronos-t5-mini
# chronos-t5-small
# chronos-t5-base
# chronos-t5-large
# chronos-bolt-tiny
# chronos-bolt-mini
# chronos-bolt-small
# chronos-bolt-base


version = "chronos-bolt-base"

pipeline = BaseChronosPipeline.from_pretrained(
    f"amazon/{version}",  # use "amazon/chronos-bolt-small" for the corresponding Chronos-Bolt model
    device_map="cuda",  # use "cpu" for CPU inference
    torch_dtype=torch.bfloat16,
)

####################################################3

import pandas as pd

import os
import csv

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados

#################################]




# product = 'glp'
# estado = 'sp'
horizon = 12



products = sorted([name for name in os.listdir('../uf/') if os.path.isdir(os.path.join('../uf/', name))])

for ano in range(2024, 2019, -1):
    periodo = (2025 - ano) * 12

    for product in products:
        folder_path = f'../uf/{product}/'
        # Read the CSV files and extract estado names
        estados = read_csv_files(folder_path)
        for estado in estados:

            df = pd.read_csv(f"../uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
            df = pd.DataFrame(df)
            df2=df
            df = df.iloc[:-periodo]


            # context must be either a 1D tensor, a list of 1D tensors,
            # or a left-padded 2D tensor with batch as the first dimension
            # quantiles is an fp32 tensor with shape [batch_size, prediction_length, num_quantile_levels]
            # mean is an fp32 tensor with shape [batch_size, prediction_length]
            quantiles, mean = pipeline.predict_quantiles(
                context=torch.tensor(df['m3']),
                prediction_length=12,
                quantile_levels=[0.1, 0.5, 0.9],
            )

            low, median, high = quantiles[0, :, 0], quantiles[0, :, 1], quantiles[0, :, 2]

            ########################################################################

            Valores_Reais = df2['m3'].tail(horizon)
            Valores_Reais = Valores_Reais.reset_index(drop=True)



            basepredictions = baseline_persistent_window(df['m3'], horizon)
            predictions_df2 = pd.DataFrame()
            predictions_df2['Predictions'] = median


            rmse_result2 = rmse(Valores_Reais, predictions_df2['Predictions'])
            mape_result2 = mape(Valores_Reais, predictions_df2['Predictions'])
            pocid_result2 = pocid(Valores_Reais, predictions_df2['Predictions'])
            mcpm_result2 = mcpm(rmse_result2, mape_result2, pocid_result2)
            pbe_result2 = pbe(Valores_Reais, predictions_df2['Predictions'])
            mase_result2 = mase(Valores_Reais, predictions_df2['Predictions'], basepredictions)


            # CSV Output VALORES REAIS
            with open(f'01-Chronos_ZS_{version}.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([
                    product, estado, 'Chronos', version, horizon, ano, mape_result2, pocid_result2, pbe_result2, mase_result2, *predictions_df2['Predictions'].values
                ])




/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  result = mae_pred/mae_naive
/tmp/ipykernel_249978/830202334.py:26: RuntimeWarning: divide by zero encountered in scalar divid